In [1]:
import os
import torch
import pandas as pd
import random
from transformers import AutoTokenizer, AutoConfig, AutoModelForPreTraining

fpath = "C:\\Users\\shafna\\Desktop\\demo_backend\\final_dataset.xlsx"
df = pd.read_excel(fpath, sheet_name ='relationship', na_filter=False)
df1 = pd.read_excel("C:\\Users\\shafna\\Desktop\\demo_backend\\UserQuestions.xlsx", sheet_name='Sheet1', na_filter=False)

C:\Users\shafna\Anaconda3\envs\rasa\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df.columns.values.tolist()

['Card', 'light_phrase', 'shadow_phrase', 'relationship_advise']

In [13]:

        if torch.cuda.is_available():
            device = torch.device('cuda')
        else:
            device = torch.device('cpu')

        model_fpath = "C:\\Users\\shafna\\Desktop\\demo_backend\\actions\\hugging_face_module\\gpt2_text_generated_model_10_10_22.bin"

        
        # Defining model configuration and hyperparameters ---------------------

        MAXLEN          = 1024  #{768, 1024, 1280, 1600}

        MODEL           = 'gpt2'

        SPECIAL_TOKENS  = {"bos_token": "<|BOS|>",
                                "eos_token": "<|EOS|>",
                                "unk_token": "<|UNK|>",
                                "pad_token": "<|PAD|>",
                                "sep_token": "<|SEP|>"}
        # ----------------------------------------------------------------------

        print('\nLoading up the model...')
        # Instantiate tokenizer
        tokenizer = AutoTokenizer.from_pretrained(MODEL)
        tokenizer.add_special_tokens(SPECIAL_TOKENS)

        # Instantiate GPT2 Model
        # if SPECIAL_TOKENS:
        config = AutoConfig.from_pretrained(MODEL,
                                            bos_token_id = tokenizer.bos_token_id,
                                            eos_token_id = tokenizer.eos_token_id,
                                            sep_token_id = tokenizer.sep_token_id,
                                            pad_token_id = tokenizer.pad_token_id,
                                            output_hidden_states=False)
        # else:
        #     config = AutoConfig.from_pretrained(MODEL,
        #                                         pad_token_id=tokenizer.eos_token_id,
        #                                         output_hidden_states=False)

        model = AutoModelForPreTraining.from_pretrained(MODEL, config=config)

        # if SPECIAL_TOKENS:
        #Special tokens added, model needs to be resized accordingly
        model.resize_token_embeddings(len(tokenizer))

        # if model_fpath:
        model.load_state_dict(torch.load(model_fpath, map_location=torch.device('cpu')))

        if torch.cuda.is_available():
            model.cuda()
        else:
            model.cpu()
        print('Loading complete.\n')
        
        def read_data():
            data = dict()
            # df = df.iloc[:144]
            for idx, row in df.iterrows():
                row['phrase'] = row['light_phrase'] + ' ' + row['shadow_phrase'] + ' ' + row['relationship_advise']
                data[idx] = [row['Card'], row['phrase']]
            print(f"Number of entries in data: {len(data) :,}")
            return data
    # fetching card and phrases fro read_data()
        for i in range(len(df)):
                data = read_data()
                index = df.index
                for i in index:
                    idx = i
                    card_name = data[idx][0]
                    phrase_text = data[idx][1]
                    for i in range(len(df1)):
                        #print(df1.loc[i, "Question"])
                             #fetching question from df1

                            sample_question = df1.loc[i, "Question"]

                            prompt = SPECIAL_TOKENS['bos_token'] + sample_question + \
                                     SPECIAL_TOKENS['sep_token'] + phrase_text + SPECIAL_TOKENS['sep_token']

                            generated = torch.tensor(tokenizer.encode(prompt[:MAXLEN])).unsqueeze(0)
                            generated = generated.to(device)

                            model.eval()

                            sample_outputs = model.generate(generated, do_sample=True, top_k=0,
                                                                 max_length=MAXLEN, num_beams=5,
                                                                 repetition_penalty=5.0,
                                                                 early_stopping=True,
                                                                 num_return_sequences=1)
                            print("{}\nSelected Tarot Card:\t{}\nQuestion:\t{}\nPhrase_text:\t{}".format('_'*70, card_name, sample_question,phrase_text, '_'*70))
                            #print("Prompt given to model:\n{}\n{}\nQuestion: \t\nphrase_text: \t\n{}\n".format(card_name, sample_question, phrase_text, '-' * 70))
                            #model_output = []   
                            for i, sample_output in enumerate(sample_outputs):
                                    text = tokenizer.decode(sample_output, skip_special_tokens=True)
                                    a = len(sample_question) + len(phrase_text)
                                    print("{}: {}\n\n".format(i+1,  text[a:]))
                            dframe = pd.read_excel('C:\\Users\\shafna\\Desktop\\demo_backend\\tarot_responses.xlsx')
                            new_row = pd.DataFrame ({'CARDS':[card_name],'QUESTIONS':[sample_question], 'PHRASES' : [phrase_text],'ANSWERS':[text[a:]]}, index=[0])
                            df2 = pd.concat([new_row,dframe.loc[:]]).reset_index(drop=True)
                            df2.to_excel("C:\\Users\\shafna\\Desktop\\demo_backend\\tarot_responses.xlsx", index=False)
        print("finished")     


Loading up the model...
Loading complete.

Number of entries in data: 22
______________________________________________________________________
Selected Tarot Card:	the fool
Question:	will my ex come back
Phrase_text:	Freeing yourself from limitation. Expressing joy and youthful vigor. Being open-minded. Taking a leap of faith. Attuning yourself to your instincts. Being eager or curious. Exploring your potential. Embracing innovation and change Being gullible and naive. Taking unnecessary risks. Failing to be serious when required. Being silly or distracted. Lacking experience. Failing to honor well-established traditions and limits. Behaving inappropriately There's a reason we say we "fall" in love! Each new romance makes us as giddy as a total beginner. Don't allow old habits to smother opportunities for growth. You can break rules without resorting to rash action—and you don't have to be in control all the time.
1: Yes your ex is likely to come back.You both are about to enter a tr

______________________________________________________________________
Selected Tarot Card:	the fool
Question:	my boyfriend is because short with me  is he still in love with me
Phrase_text:	Freeing yourself from limitation. Expressing joy and youthful vigor. Being open-minded. Taking a leap of faith. Attuning yourself to your instincts. Being eager or curious. Exploring your potential. Embracing innovation and change Being gullible and naive. Taking unnecessary risks. Failing to be serious when required. Being silly or distracted. Lacking experience. Failing to honor well-established traditions and limits. Behaving inappropriately There's a reason we say we "fall" in love! Each new romance makes us as giddy as a total beginner. Don't allow old habits to smother opportunities for growth. You can break rules without resorting to rash action—and you don't have to be in control all the time.
1: You are pure soul.Your partner has alot on their plate regarding their career, family, or even 

______________________________________________________________________
Selected Tarot Card:	the fool
Question:	dose Gerado love me will we ger back together in time for my party what dose are future look like  together
Phrase_text:	Freeing yourself from limitation. Expressing joy and youthful vigor. Being open-minded. Taking a leap of faith. Attuning yourself to your instincts. Being eager or curious. Exploring your potential. Embracing innovation and change Being gullible and naive. Taking unnecessary risks. Failing to be serious when required. Being silly or distracted. Lacking experience. Failing to honor well-established traditions and limits. Behaving inappropriately There's a reason we say we "fall" in love! Each new romance makes us as giddy as a total beginner. Don't allow old habits to smother opportunities for growth. You can break rules without resorting to rash action—and you don't have to be in control all the time.
1: Everything appears to go good till now.You both might 

______________________________________________________________________
Selected Tarot Card:	the magician
Question:	Someday Tamara will return to my life?
Phrase_text:	Taking appropriate action. Receiving guidance from a higher power. Becoming a channel of divine will. Expressing masculine energy in appropriate and constructive ways. Being yourself in every way. Inflating your own ego. Abusing talents. Manipulating or deceiving others. Being too aggressive. Using cheap illusions to dazzle others. Refusing to invest the time and effort needed to master your craft. Taking shortcuts. A relationship should empower you. Does yours generate positive change in your life? Do you feel happier and more fulfilled? A relationship that supports your goals is to be valued; a relationship that doesn't must be changed or cast aside.
1: You are a pure soul.Your partner has alot of love and devotion for you.They want to start a family with you. 


_________________________________________________________

______________________________________________________________________
Selected Tarot Card:	the magician
Question:	I am concerned About my fiancé and I 
Phrase_text:	Taking appropriate action. Receiving guidance from a higher power. Becoming a channel of divine will. Expressing masculine energy in appropriate and constructive ways. Being yourself in every way. Inflating your own ego. Abusing talents. Manipulating or deceiving others. Being too aggressive. Using cheap illusions to dazzle others. Refusing to invest the time and effort needed to master your craft. Taking shortcuts. A relationship should empower you. Does yours generate positive change in your life? Do you feel happier and more fulfilled? A relationship that supports your goals is to be valued; a relationship that doesn't must be changed or cast aside.
1: You are a pure soul.Your partner has alot on their plate regarding their career, family, or even their mental health.They may seem intriguing at first but can get into tr

...

______________________________________________________________________
Selected Tarot Card:	the lovers
Question:	Is my husband cheating on me? Twanita Peterson  Sarasota FL
Phrase_text:	Being in love. Showing your love to others. Expressing passion or romantic feelings. Aligning yourself with groups or like-minded others. Bringing people together. Making well-informed decisions. Debilitating passion. Allowing an unhealthy desire for love to motivate destructive behavior. Disrupting unity. Working against the best interests of those who care about you. Ill-informed decisions. This is a time to celebrate unity, love, and romance in all its forms. Souls are coming together, and a unique opportunity exists for people to enjoy an unusual degree of interdependence and cooperation. Let your heart guide your choices today.
1: Yes your partner is cheating on you.They have alot on their plate regarding their career, family, or even their mental health.Take action if thats not what you want.


__

______________________________________________________________________
Selected Tarot Card:	the chariot
Question:	will my ex come back
Phrase_text:	Breaking through barriers. Moving forward with confidence and authority. Reaching the pinnacle of success. Basking in the glory of achievement. Guiding an effort to total victory. Establishing yourself as a worthy leader. Resting on laurels. Riding roughshod over the feelings or expectations of others. Focusing more on past successes than future opportunities. Failing to rein in impulsive behavior. Moving forward to mutual victory requires trust and cooperation. If you have this already, depend on it during an upcoming challenge. If you lack it, consider what choices have prevented forming this kind of bond. Don't rest on your laurels  successful romance is an ongoing commitment.
1: Yes your ex will applogise to you soon.They are trustworthy and wants to start a family with you.


____________________________________________________________

______________________________________________________________________
Selected Tarot Card:	the chariot
Question:	will Gary and I be together
Phrase_text:	Breaking through barriers. Moving forward with confidence and authority. Reaching the pinnacle of success. Basking in the glory of achievement. Guiding an effort to total victory. Establishing yourself as a worthy leader. Resting on laurels. Riding roughshod over the feelings or expectations of others. Focusing more on past successes than future opportunities. Failing to rein in impulsive behavior. Moving forward to mutual victory requires trust and cooperation. If you have this already, depend on it during an upcoming challenge. If you lack it, consider what choices have prevented forming this kind of bond. Don't rest on your laurels  successful romance is an ongoing commitment.
1: You both are blessed souls.The relationship will bring alot of stability and prosperity to your life.


_________________________________________________

______________________________________________________________________
Selected Tarot Card:	the chariot
Question:	my husband is he going to stay and love us or is he leaving
Phrase_text:	Breaking through barriers. Moving forward with confidence and authority. Reaching the pinnacle of success. Basking in the glory of achievement. Guiding an effort to total victory. Establishing yourself as a worthy leader. Resting on laurels. Riding roughshod over the feelings or expectations of others. Focusing more on past successes than future opportunities. Failing to rein in impulsive behavior. Moving forward to mutual victory requires trust and cooperation. If you have this already, depend on it during an upcoming challenge. If you lack it, consider what choices have prevented forming this kind of bond. Don't rest on your laurels  successful romance is an ongoing commitment.
1: You are likely to find that your partner will not be happy where you are right now.


___________________________________

KeyboardInterrupt: 

In [11]:
# index = df.index
# for i in index:
#     idx = i
#     print(idx)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [4]:
# for i in range(len(df1)):
#     #print(df1.loc[i, "Question"])
#          #fetching question from df1
#         sample_question = df1.loc[i, "Question"]

#         prompt = SPECIAL_TOKENS['bos_token'] + sample_question + \
#                  SPECIAL_TOKENS['sep_token'] + phrase_text + SPECIAL_TOKENS['sep_token']

#         generated = torch.tensor(tokenizer.encode(prompt[:MAXLEN])).unsqueeze(0)
#         generated = generated.to(device)
#         model.eval()

#         sample_outputs = model.generate(generated, do_sample=True, top_k=0,
#                                              max_length=MAXLEN, num_beams=5,
#                                              repetition_penalty=5.0,
#                                              early_stopping=True,
#                                              num_return_sequences=1)
#         #print("{}\nSelected Tarot Card:\t{}\nQuestion:\t{}\nPhrase_text:\t{}".format('_'*70, card_name, sample_question,phrase_text, '_'*70))
#         #print("Prompt given to model:\n{}\n{}\nQuestion: \t\nphrase_text: \t\n{}\n".format(card_name, sample_question, phrase_text, '-' * 70))
#         #model_output = []   
#         for i, sample_output in enumerate(sample_outputs):
#                 text = tokenizer.decode(sample_output, skip_special_tokens=True)
#                 a = len(sample_question) + len(phrase_text)
#                 #print("{}: {}\n\n".format(i+1,  text[a:]))
#         dframe = pd.read_excel('C:\\Users\\shafna\\Desktop\\demo_backend\\tarot_responses.xlsx')
#         new_row = pd.DataFrame ({'CARDS':[card_name],'QUESTIONS':[sample_question], 'PHRASES' : [phrase_text],'ANSWERS':[text[a:]]}, index=[0])
#         df2 = pd.concat([new_row,dframe.loc[:]]).reset_index(drop=True)
#         df2.to_excel("C:\\Users\\shafna\\Desktop\\demo_backend\\tarot_responses.xlsx", index=False)